In [37]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import scipy.ndimage as sn

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure
import warnings

In [38]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [39]:
name_project = 'project/output_FJSJ_17-01/'               
#name_project = 'project_repartition_v3.0/output_repar_v9.1_02-01/'               
name_project

'project/output_FJSJ_17-01/'

In [40]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/


In [41]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [42]:
key_subworks = info_basic['key_subworks']
key_subworks

['04-02',
 '05-02',
 '06-02',
 '07-02',
 '08-02',
 '09-02',
 '10-02',
 '11-02',
 '12-02',
 '13-02',
 '14-02',
 '15-02',
 '16-02',
 '17-02',
 '18-02',
 '19-02',
 '20-02',
 '21-02',
 '22-02',
 '23-02',
 '24-02',
 '25-02',
 '26-02',
 '27-02',
 '28-02',
 '29-02',
 '30-02',
 '31-02',
 '32-02',
 '33-02',
 '34-02',
 '35-02',
 '36-02',
 '37-02',
 '38-02',
 '39-02',
 '40-02',
 '41-02',
 '42-02',
 '43-02',
 '44-02',
 '45-02',
 '46-02',
 '47-02',
 '48-02',
 '49-02',
 '50-02',
 '51-02',
 '52-02',
 '53-02',
 '54-02',
 '55-02',
 '56-02',
 '57-02',
 '04-03',
 '05-03',
 '06-03',
 '07-03',
 '08-03',
 '09-03',
 '10-03',
 '11-03',
 '12-03',
 '13-03',
 '14-03',
 '15-03',
 '16-03',
 '17-03',
 '18-03',
 '19-03',
 '20-03',
 '21-03',
 '22-03',
 '23-03',
 '24-03',
 '25-03',
 '26-03',
 '27-03',
 '28-03',
 '29-03',
 '30-03',
 '31-03',
 '32-03',
 '33-03',
 '34-03',
 '35-03',
 '36-03',
 '37-03',
 '38-03',
 '39-03',
 '40-03',
 '41-03',
 '42-03',
 '43-03',
 '44-03',
 '45-03',
 '46-03',
 '47-03',
 '48-03',
 '49-03',


In [43]:
dir_image = dir_project+info_basic['dir_image'] + 'autopick/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_ds = dir_project+info_basic['dir_ds']
print('dir_image: ', dir_image)
print('dir_ds: ', dir_ds)

dir_image:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/image_17-01/autopick/
dir_ds:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/ds_17-01/


In [44]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/autopick17-01//


In [45]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [46]:
ds_remove_all = {}
for key_subwork in key_subworks:
    print(key_subwork)
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds.close()
    ds_remove_all[key_subwork] = ds_remove

04-02
05-02
06-02
07-02
08-02
09-02
10-02
11-02
12-02
13-02
14-02
15-02
16-02
17-02
18-02
19-02
20-02
21-02
22-02
23-02
24-02
25-02
26-02
27-02
28-02
29-02
30-02
31-02
32-02
33-02
34-02
35-02
36-02
37-02
38-02
39-02
40-02
41-02
42-02
43-02
44-02
45-02
46-02
47-02
48-02
49-02
50-02
51-02
52-02
53-02
54-02
55-02
56-02
57-02
04-03
05-03
06-03
07-03
08-03
09-03
10-03
11-03
12-03
13-03
14-03
15-03
16-03
17-03
18-03
19-03
20-03
21-03
22-03
23-03
24-03
25-03
26-03
27-03
28-03
29-03
30-03
31-03
32-03
33-03
34-03
35-03
36-03
37-03
38-03
39-03
40-03
41-03
42-03
43-03
44-03
45-03
46-03
47-03
48-03
49-03
50-03
51-03
52-03
53-03
54-03
55-03
56-03
57-03
04-04
05-04
06-04
07-04
08-04
09-04
10-04
11-04
12-04
13-04
14-04
15-04
16-04
17-04
18-04
19-04
20-04
21-04
22-04
23-04
24-04
25-04
26-04
27-04
28-04
29-04
30-04
31-04
32-04
33-04
34-04
35-04
36-04
37-04
38-04
39-04
40-04
41-04
42-04
43-04
44-04
45-04
46-04
47-04
48-04
49-04
50-04
51-04
52-04
53-04
54-04
55-04
56-04
57-04
04-05
05-05
06-05
07-05
08-0

### Pick probes' Phase velocity automatically

In [47]:
def cluster_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  
    ds = np.nan_to_num(ds)
    for i in range(ds.shape[1]):
        ds[:,i] = ds[:,i]/np.max(ds[:,i])

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [48]:
def pick_fundamental(key):
    global f
    global c
    global f_new
    global c_new
    global flag_f
    global flag_c
    global dir_ds
    global threshold
    global dir_disp
    global dir_ds
    global info_basic
    global info_basic_bi
    global smooth_cluster
    global smooth_pick
    global key_subworks
    global ds_remove_all

    
    ds_remove = ds_remove_all[key]
    grace = 0.0

    f_start = flag_f[0]
    f_end = flag_f[1]
    c_min = flag_c[0]
    c_max = flag_c[1]

    ff_all = []
    cc_all = []


    while f_start != f_end:
        #print(f_start,f_end)
        f_new = f[np.logical_and(f>f_start,f <= f_end)]
        c_new = c[np.logical_and(c>c_min,c < c_max)]
        ds_new0 = ds_remove[np.logical_and(c>c_min,c < c_max),:]
        ds_new = ds_new0[:,np.logical_and(f>f_start,f <= f_end)]
        # 对数组进行横向的平滑
        ds_new = pd.DataFrame(ds_new)
        #ds_new = ds_new.rolling(100).mean()
        ds_new = ds_new.T
        ds_new = ds_new.rolling(smooth_cluster).mean()
        ds_new = np.array(ds_new)
        ds_new = ds_new.T
        ff,cc = cluster_fundamental(ds_new)
        if len(ff_all) == 0:
            ff_all = ff
            cc_all = cc
        elif len(ff) != 0:
            ff_all = np.hstack((ff_all,ff))
            cc_all = np.hstack((cc_all,cc))
        else:
            #print('this')
            break
        f_start = np.max(ff)
        c_max = np.max(cc[ff==f_start]) + grace
    
    # smooth
    # 转置
    """
    ds_smooth = ds_remove.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    ds_smooth = ds_smooth.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    """
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = smooth_pick,level_c = smooth_pick)

    f_range = set(ff_all)
    f_range = np.array(list(f_range))
    c_pick = []
    for i in range(len(f_range)):
        f_this = f_range[i]
        c_this = cc_all[ff_all==f_this]
        c_range = [np.min(c_this),np.max(c_this)]
        c_new = c[np.logical_and(c>=c_range[0],c<=c_range[1])]
        c_pick.append(c_new[np.argmax(ds_smooth[np.logical_and(c>=c_range[0],c<=c_range[1]),f==f_this])])
    c_pick = np.array(c_pick)
    # smooth
    index = np.argsort(f_range)
    f_range = f_range[index]
    c_pick = c_pick[index]
    c_pick_smooth = sn.gaussian_filter1d(c_pick,6)
    
    # save as txt
    np.savetxt(dir_disp+'autopick_cluster_'+key+'.txt', np.array([ff_all,cc_all]).T, fmt='%.4f', delimiter=',')
    np.savetxt(dir_disp+'autopick_'+key+'.txt', np.array([f_range,c_pick,c_pick_smooth]).T, fmt='%.4f', delimiter=',')
    print('finish '+ key + ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))

In [49]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
flag_c = [0.25,1.1]
flag_f = [7.5,20]
#flag_f = [6,25]
threshold = 0.9
f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]
smooth_cluster = 30
smooth_pick = 6
flag_pick = 1

In [50]:
warnings.filterwarnings("ignore")
if flag_pick == 1:
    for key_subwork in key_subworks:
        pick_fundamental(key_subwork)

finish 04-02 1/972
finish 05-02 2/972
finish 06-02 3/972
finish 07-02 4/972
finish 08-02 5/972
finish 09-02 6/972
finish 10-02 7/972
finish 11-02 8/972
finish 12-02 9/972
finish 13-02 10/972
finish 14-02 11/972
finish 15-02 12/972
finish 16-02 13/972
finish 17-02 14/972
finish 18-02 15/972
finish 19-02 16/972
finish 20-02 17/972
finish 21-02 18/972
finish 22-02 19/972
finish 23-02 20/972
finish 24-02 21/972
finish 25-02 22/972
finish 26-02 23/972
finish 27-02 24/972
finish 28-02 25/972
finish 29-02 26/972
finish 30-02 27/972
finish 31-02 28/972
finish 32-02 29/972
finish 33-02 30/972
finish 34-02 31/972
finish 35-02 32/972
finish 36-02 33/972
finish 37-02 34/972
finish 38-02 35/972
finish 39-02 36/972
finish 40-02 37/972
finish 41-02 38/972
finish 42-02 39/972
finish 43-02 40/972
finish 44-02 41/972
finish 45-02 42/972
finish 46-02 43/972
finish 47-02 44/972
finish 48-02 45/972
finish 49-02 46/972
finish 50-02 47/972
finish 51-02 48/972
finish 52-02 49/972
finish 53-02 50/972
finish 54

In [51]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image
    global key_subworks

    fig,ax = plt.subplots(1,3,figsize=(18,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ff_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    c_smooth = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,2]

    ax[0] = plotlib.plot_fj(ax[0],ds_remove,title0,f,c,0,c_map='jet')
    ax[0].scatter(ff_all,cc_all,marker='o',s=10,color='k')
    
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = 10,level_c=6)


    ax[1] = plotlib.plot_fj(ax[1],ds_smooth,title0,f,c,0,c_map='jet')
    ax[1].scatter(f_pick,c_pick,marker='o',s=10,color='k')

    ax[2] = plotlib.plot_fj(ax[2],ds_smooth,title0,f,c,0,c_map='jet')
    ax[2].scatter(f_pick,c_smooth,marker='o',s=10,color='k')

    plt.savefig(dir_image+'autopick_'+key_subwork+'.png',dpi=60)
    plt.tight_layout()
    plt.close()
    print('finish '+key_subwork,' ', str(key_subworks.index(key_subwork)+1),'/',str(len(key_subworks)))

In [52]:
flag_plot = 1

In [53]:
if flag_plot == 1:
    for key_subwork in key_subworks:
        plot_disp(key_subwork)

finish 04-02   1 / 972
finish 05-02   2 / 972
finish 06-02   3 / 972
finish 07-02   4 / 972
finish 08-02   5 / 972
finish 09-02   6 / 972
finish 10-02   7 / 972
finish 11-02   8 / 972
finish 12-02   9 / 972
finish 13-02   10 / 972
finish 14-02   11 / 972
finish 15-02   12 / 972
finish 16-02   13 / 972
finish 17-02   14 / 972
finish 18-02   15 / 972
finish 19-02   16 / 972
finish 20-02   17 / 972
finish 21-02   18 / 972
finish 22-02   19 / 972
finish 23-02   20 / 972
finish 24-02   21 / 972
finish 25-02   22 / 972
finish 26-02   23 / 972
finish 27-02   24 / 972
finish 28-02   25 / 972
finish 29-02   26 / 972
finish 30-02   27 / 972
finish 31-02   28 / 972
finish 32-02   29 / 972
finish 33-02   30 / 972
finish 34-02   31 / 972
finish 35-02   32 / 972
finish 36-02   33 / 972
finish 37-02   34 / 972
finish 38-02   35 / 972
finish 39-02   36 / 972
finish 40-02   37 / 972
finish 41-02   38 / 972
finish 42-02   39 / 972
finish 43-02   40 / 972
finish 44-02   41 / 972
finish 45-02   42 / 972
f